# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2022-11-16'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2506,2022-11-11,"$17,583.25","$17,650.94","$16,543.48","$17,034.29","$17,034.29",55871616488
2507,2022-11-12,"$17,036.88","$17,066.68","$16,651.78","$16,799.19","$16,799.19",29717699419
2508,2022-11-13,"$16,799.72","$16,920.77","$16,320.63","$16,353.37","$16,353.37",27209183682
2509,2022-11-14,"$16,352.03","$17,109.32","$15,872.94","$16,618.20","$16,618.20",49630243054
2510,2022-11-15,"$16,617.48","$17,051.96","$16,542.55","$16,884.61","$16,884.61",36599436183


In [ ]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()


Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = ["Date","Open"]
btc_df =btc_df[df]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para rembrar las columnas de df
# df = btc_df[["ds","y"]]
df = btc_df.rename(columns={'Date':'ds','Open':'y'})


In [ ]:
df

,ds,y
0,2016-01-01,$430.72
1,2016-01-02,$434.62
2,2016-01-03,$433.58
3,2016-01-04,$430.06
4,2016-01-05,$433.07
...,...,...
2506,2022-11-11,"$17,583.25"
2507,2022-11-12,"$17,036.88"
2508,2022-11-13,"$16,799.72"
2509,2022-11-14,"$16,352.03"


In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1798,2022-10-12 00:00:00+00:00,"$1,279.73"
1799,2022-10-13 00:00:00+00:00,"$1,294.92"
1800,2022-10-14 00:00:00+00:00,"$1,288.05"
1801,2022-10-15 00:00:00+00:00,"$1,297.31"
1802,2022-10-16 00:00:00+00:00,"$1,275.01"


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
# Python
m= Prophet(seasonality_mode="multiplicative")


In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()


,ds
2871,2023-11-11
2872,2023-11-12
2873,2023-11-13
2874,2023-11-14
2875,2023-11-15


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future = future[ future['ds'].dt.dayofweek < 5 ]

<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)



In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2049,2023-11-09,"$-13,412.06","$-43,295.71","$12,432.73"
2050,2023-11-10,"$-13,466.66","$-43,408.39","$11,440.36"
2051,2023-11-13,"$-13,763.35","$-43,556.58","$11,555.94"
2052,2023-11-14,"$-13,857.12","$-43,161.15","$11,923.19"
2053,2023-11-15,"$-13,933.79","$-44,661.79","$12,323.76"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)